In [1]:
!pip install nbformat nbconvert


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: C:\Users\Usuario\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import mysql.connector
from mysql.connector import Error
import os
from dotenv import load_dotenv
import pandas as pd
from sqlalchemy import create_engine, inspect


In [3]:
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor

notebook_path_to_execute = './4_delete_tables_in_aiven.ipynb'

def ejecutar_notebook(notebook_path):
    try:
        with open(notebook_path, encoding='utf-8') as f:
            nb = nbformat.read(f, as_version=4)

        ep = ExecutePreprocessor(timeout=600, kernel_name='python3')

        ep.preprocess(nb, {'metadata': {'path': './'}})
        print(f'✅ Execution {notebook_path}')

    except UnicodeDecodeError as e:
        print(f"❌ Error decoding in {notebook_path}: {e}")
    except Exception as e:
        print(f'❌ Error during execution {notebook_path}: {e}')


ejecutar_notebook(notebook_path_to_execute)
    

✅ Execution ./4_delete_tables_in_aiven.ipynb


In [4]:
load_dotenv()

host = os.getenv('host')
port = os.getenv('port')
database = os.getenv('database')
user = os.getenv('user')
password = os.getenv('password')


In [5]:
dir_path = '../data/data_processed/'

In [6]:
def get_name(name):
    return os.path.splitext(name)[0]

In [7]:
summary = []


In [8]:
try:
    connection = mysql.connector.connect(
        user=user,
        password=password,
        host=host,
        port=port,
        database=database
    )

    if connection.is_connected():
        print("✅ Connected to database")

        for archivo in os.listdir(dir_path):
            if archivo.endswith('.csv'):
                file_path = os.path.join(dir_path, archivo)
                
                try:
                    df = pd.read_csv(file_path, sep=',', encoding='utf-8')
                    print(f"\n\n📄 Processing file: {archivo}")
                    print(df.head())
                    print(df.info())

                    table_name = get_name(archivo)

                    df = df.where(pd.notnull(df), None)  

                    cursor = connection.cursor()

                    columns = ', '.join(f"`{col}` VARCHAR(255)" for col in df.columns)
                    primary_key_column = 'index'
                    create_table_query = f"CREATE TABLE IF NOT EXISTS `{table_name}` ({columns}, PRIMARY KEY (`{primary_key_column}`))"
                    cursor.execute(create_table_query)
                    print(f"🛠️ Table '{table_name}' created or already exists.")

                    try:
                        print(f"⚙️ Setting data in '{table_name}'...")

                        for i, row in enumerate(df.itertuples(index=False), 1):
                            insert_query = f"INSERT INTO `{table_name}` (`{'`, `'.join(df.columns)}`) VALUES ({', '.join(['%s']*len(df.columns))})"
                            cursor.execute(insert_query, tuple(row))

                            if i % 10 == 0:
                                print(f"Inserted rows {i}")

                        connection.commit()
                        print(f"✨ All data inserted in '{table_name}'")
                        summary.append((table_name, '✅'))

                    except Error as e:
                        print("❌ Error during insert data:", e)
                        summary.append((table_name, f"❌ Error during insert data: {e}"))

                except Error as e:
                    print(f"❌ Error during processing file {archivo}: ", e)
                    summary.append((table_name, f"❌ Error during processing file: {e}"))

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("🔒 MySQL connection closed")

print("\nSummary:")
for table, status in summary:
    print(f"Table: {table} - Status: {status}")

✅ Connected to database


📄 Processing file: MunE19_c_mod_comarcas.csv
   index           TH  Cod Comarca  Desempleo
0      0  Araba-Alava            1       0.09
1      1  Araba-Alava            2       0.10
2      2  Araba-Alava            3       0.11
3      3  Araba-Alava           99       0.05
4      4      Bizkaia            1       0.08
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   index        14 non-null     int64  
 1   TH           14 non-null     object 
 2   Cod Comarca  14 non-null     int64  
 3   Desempleo    14 non-null     float64
dtypes: float64(1), int64(2), object(1)
memory usage: 580.0+ bytes
None
🛠️ Table 'MunE19_c_mod_comarcas' created or already exists.
⚙️ Setting data in 'MunE19_c_mod_comarcas'...
Inserted rows 10
✨ All data inserted in 'MunE19_c_mod_comarcas'


📄 Processing file: MunE19_c_mod_municipios.csv
   index

In [9]:
connection_str = f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}"

try:
    engine = create_engine(connection_str)
    connection = engine.connect()

    print("✅ Successfully connected to the database")

    inspector = inspect(engine)

    tables = inspector.get_table_names()
    print(f"🔍 Tables found: {tables}")

    for table_name in tables:
        try:
            query = f"SELECT * FROM `{table_name}`"
            df = pd.read_sql(query, con=engine)

            print(f"\n🔍 Table '{table_name}':")
            print(df.head())
            print(df.info())
        except Exception as e:
            print(f"❌ Error querying table '{table_name}': {e}")

except Exception as e:
    print(f"❌ Error connecting to the database: {e}")

finally:
    if 'connection' in locals() and connection is not None:
        connection.close()
        print("🔒 MySQL connection closed")


✅ Successfully connected to the database
🔍 Tables found: ['MunE19_c_mod_comarcas', 'MunE19_c_mod_municipios', 'MunE19_c_mod_partidos_sesgo', 'MunE19_c_mod_resultados_municipios', 'MunE19_c_mod_votos_partido_municipios']

🔍 Table 'MunE19_c_mod_comarcas':
  index           TH Cod Comarca Desempleo
0     0  Araba-Alava           1      0.09
1     1  Araba-Alava           2       0.1
2    10     Gipuzkoa           2      0.11
3    11     Gipuzkoa           3      0.13
4    12     Gipuzkoa          40       0.1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   index        14 non-null     object
 1   TH           14 non-null     object
 2   Cod Comarca  14 non-null     object
 3   Desempleo    14 non-null     object
dtypes: object(4)
memory usage: 580.0+ bytes
None

🔍 Table 'MunE19_c_mod_municipios':
  index           TH Cod Comarca Cod Municipio        